## Step 1. 安装所需依赖

In [ ]:
!pip install -r requirements.txt

## Step 2. 从ModelScope下载模型

选取的模型为 Qwen3-30B-A3B-Instruct-2507

In [ ]:
!modelscope download --model Qwen/Qwen3-30B-A3B-Instruct-2507  --local_dir ./Qwen/

In [ ]:
# 测试模型推理
import os
# 删除:from modelscope.pipelines import pipeline
# 删除:from modelscope.utils.constant import Tasks

# 添加vLLM相关导入
from vllm import LLM
from vllm.sampling_params import SamplingParams

# 设置模型路径（请根据实际下载路径进行调整）
model_path = './Qwen/Qwen3-30B-A3B-Instruct-2507'

# 检查模型路径是否存在
if not os.path.exists(model_path):
    raise FileNotFoundError(f"模型路径不存在: {model_path}")

# 创建vLLM模型实例
llm = LLM(
    model=model_path,
    trust_remote_code=True
)

# 启动推理服务
def infer(text):
    # 使用vLLM进行推理
    sampling_params = SamplingParams(temperature=0.7, max_tokens=512)
    outputs = llm.generate([text], sampling_params)
    return outputs[0].outputs[0].text

# 测试模型推理
if __name__ == '__main__':
    test_input = "你好，世界！"
    output = infer(test_input)
    print("模型输出:", output)

In [ ]:
# 销毁模型
import gc
import torch
del llm
gc.collect()
torch.cuda.empty_cache()


## Step 3. 部署模型为推理服务

这里使用Fast API方式部署，持久化推荐vllm部署

In [ ]:
# 加载模型服务
!python server.py --model './Qwen/Qwen3-30B-A3B-Instruct-2507' --host 0.0.0.0 --port 8000

^C


In [ ]:
# 测试API端点
def test_api_endpoint(endpoint="/v1/models", host="127.0.0.1", port=8000, token=None):
    import requests
    url = f"http://{host}:{port}{endpoint}"
    print(f"测试API端点: {url}")
    headers = {}
    if token:
        headers["Authorization"] = f"Bearer {token}"
    try:
        response = requests.get(url, headers=headers)
        print(f"响应状态码: {response.status_code}")
        print(f"响应内容: {response.json()}")
        return response.json()
    except Exception as e:
        print(f"测试API端点时出错: {str(e)}")
        return None
import dotenv
dotenv.load_dotenv()
token = os.getenv("API_KEY")
test_api_endpoint(token=token)

In [ ]:
# 测试客户端测试推理（可忽略）
%TOKEN="sk-1234567890"
!python client.py --host 127.0.0.1 --port 8000 --token $TOKEN --model Qwen3-30B-A3B-Instruct-2507 --prompt "你好，超算互联网！"